In [1]:
#librerias
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import modulo_conn_sql as mcq
import sqlalchemy as sa
import urllib

#Query BD SQL-Server Cemex
def querySQL(query, parametros):
    #Conectar con base sql y ejecutar consulta
    cursor = conectarSQL()
    try:
        cursor.execute(query, parametros)
        #obtener nombre de columnas
        names = [ x[0] for x in cursor.description]
        
        #Reunir todos los resultado en rows
        rows = cursor.fetchall()
        resultadoSQL = []
            
        #Hacer un array con los resultados
        while rows:
            resultadoSQL.append(rows)
            if cursor.nextset():
                rows = cursor.fetchall()
            else:
                rows = None
                
        #Redimensionar el array para que quede en dos dimensiones
        resultadoSQL = np.array(resultadoSQL)
        resultadoSQL = np.reshape(resultadoSQL, (resultadoSQL.shape[1], resultadoSQL.shape[2]) )
    finally:
            if cursor is not None:
                cursor.close()
    return pd.DataFrame(resultadoSQL, columns = names)

#SQL Methods to get operation data
def conectarSQL():
    conn = mcq.ConexionSQL()
    cursor = conn.getCursor()
    return cursor

In [2]:
pais = 'Colombia'

#rangos de fecha
inicio_historia = '2022-02-01'
fin_historia = '2022-02-27'




#informacion de la programacion
df = querySQL( "SELECT * FROM SCAC_AT2_condensadoservicio where FechaEntrega BETWEEN  ? AND  ? " , (inicio_historia, fin_historia) )
df['Posicion'] = df['Posicion'].astype(int)
df['VolPartida'] = df['VolPartida'].astype(float)
df['Pedido'] = df['Pedido'].str[:10]

#agrego informacion geografica        
nombre_cluster = querySQL( "SELECT Pais, [Desc Cluster] as Cluster, [Planta Unica] as PlantaUnica, Centro FROM SCAC_AT1_NombreCluster where Pais = ?" , (pais) )

#INFORME DE VENTAS
info_ventas = pd.read_excel('./datos/IV-2.xlsx')
info_ventas['Documento Ventas'] = info_ventas['Documento Ventas'].astype(str)



#Relaciono un pedido a una planta
df1 = pd.merge(df[(df['Entrega'] != '') & (df['Estatus'] == 'Normal') & (df['Posicion'] < 3000) & (df['ClaseDocumento'] != 'UsoIntern')] , nombre_cluster, left_on='Planta', right_on='Centro', how = 'inner' )
#df1 = pd.merge(df[(df['Entrega'] != '') & (df['Posicion'] < 3000) & (df['ClaseDocumento'] != 'UsoIntern')] , nombre_cluster, left_on='Planta', right_on='Centro', how = 'inner' )
pedidos = df1.groupby(['Pais', 'Cluster', 'PlantaUnica', 'Pedido', 'FechaEntrega'])['VolPartida'].agg(['sum']).reset_index()
pedidos['rank'] = pedidos.groupby('Pedido')['sum'].rank(ascending=False, method='first')
pedidos = pedidos[pedidos['rank'] == 1]

#obtengo la suma del volumen despachado de cada pedido (quito la posicion del bombeo para no duplicar volumen)
pedidos_volumen = df1[df1['Posicion'] < 3000].groupby(['Pedido'])['VolPartida'].agg(['sum']).reset_index()

#junto ambos dataframe para obtener el volumen del pedido junto con su informacion geografica
lista_pedidos = pd.merge(pedidos, pedidos_volumen, on='Pedido').drop(['sum_x','rank'], axis = 1)

#hago un left join entre el despacho y ventas para detectar cuales pedidos estan registrados como despachados pero no facturados
df_final = pd.merge(lista_pedidos, info_ventas, left_on='Pedido', right_on='Documento Ventas', how='left').drop(['Documento Ventas'], axis=1)
df_final.rename(columns={'sum_y': 'Despachado', 'Total': 'Venta'}, inplace = True)


#despacho != facturado
despacho_distinto_facturado = df_final[(df_final['Venta'] != df_final['Despachado']) & ( df_final['Venta'].notnull() ) ]
despacho_distinto_facturado['gap'] = despacho_distinto_facturado['Despachado'] - despacho_distinto_facturado['Venta']


#FACTURADO NO DESPACHADO
#Relaciono un pedido a una planta
df2 = pd.merge(df[(df['Estatus'] != 'Normal') & (df['Posicion'] < 3000) & (df['ClaseDocumento'] != 'UsoIntern')] , nombre_cluster, left_on='Planta', right_on='Centro', how = 'inner' )
pedidos2 = df2.groupby(['Pais', 'Cluster', 'PlantaUnica', 'Pedido', 'FechaEntrega', 'Estatus'])['VolPartida'].agg(['sum']).reset_index()
pedidos2['rank'] = pedidos2.groupby('Pedido')['sum'].rank(ascending=False, method='first')
pedidos2 = pedidos2[pedidos2['rank'] == 1]

#obtengo la suma del volumen despachado de cada pedido (quito la posicion del bombeo para no duplicar volumen)
pedidos_volumen2 = df2[df2['Posicion'] < 3000].groupby(['Pedido'])['VolPartida'].agg(['sum']).reset_index()

#junto ambos dataframe para obtener el volumen del pedido junto con su informacion geografica
lista_pedidos2 = pd.merge(pedidos2, pedidos_volumen2, on='Pedido').drop(['sum_x','rank'], axis = 1)

#hago un left join entre el despacho y ventas para detectar cuales pedidos estan registrados como facturados pero no despachados
df_final2 = pd.merge(info_ventas[info_ventas['Total'] > 0], lista_pedidos[['Pedido']], right_on='Pedido', left_on='Documento Ventas', how='left')
df_final2 = df_final2[df_final2['Pedido'].isnull()].drop(['Pedido'], axis=1)

facturado_no_despachado = pd.merge( lista_pedidos2, df_final2, left_on='Pedido', right_on='Documento Ventas', how='right' ).drop(['Pedido','sum_y'], axis = 1)
facturado_no_despachado.rename(columns={'Total':'Venta', 'Documento Ventas':'Pedido'}, inplace=True)
facturado_no_despachado['Despachado'] = 0
facturado_no_despachado['Categoria'] = "Facturado no despachado"
facturado_no_despachado = facturado_no_despachado[['Pais','Cluster','PlantaUnica','Pedido', 'FechaEntrega','Venta','Categoria', 'Estatus']]


#RESULTADO FINAL
df_final['Categoria'] = "Despacho no facturado"
despacho_distinto_facturado['Categoria'] = "Despacho distinto a facturado"
df_sql = pd.concat([df_final[df_final['Venta'].isnull()], despacho_distinto_facturado[despacho_distinto_facturado['gap'] > 0]]).reset_index()
df_sql = pd.concat([df_sql,facturado_no_despachado ])
df_sql = df_sql.drop(['index','gap'], axis= 1)
df_sql

<ipython-input-2-a74a17f1a023>:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  despacho_distinto_facturado['gap'] = despacho_distinto_facturado['Despachado'] - despacho_distinto_facturado['Venta']
<ipython-input-2-a74a17f1a023>:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  despacho_distinto_facturado['Categoria'] = "Despacho distinto a facturado"


,Pais,Cluster,PlantaUnica,Pedido,FechaEntrega,Despachado,Venta,Categoria,Estatus
0,Colombia,CLUSTER ANTIOQUIA,CO-PLANTA MEDELLIN,8012259742,2022-02-02,7.00,NaN,Despacho no facturado,NaN
1,Colombia,CLUSTER CENTRO,CO-PLANTA 240,8012280248,2022-02-02,8.50,NaN,Despacho no facturado,NaN
2,Colombia,CLUSTER COSTA,CO-PLANTA BARRANQUILLA,8012265070,2022-02-02,6.00,NaN,Despacho no facturado,NaN
3,Colombia,CLUSTER SANTANDER,CO-PLANTA FLORIDA,8012272138,2022-02-02,7.25,NaN,Despacho no facturado,NaN
4,Colombia,CLUSTER SURORIENTE,CO-PLANTA FUSA,8012370141,2022-02-23,4.00,NaN,Despacho no facturado,NaN
...,...,...,...,...,...,...,...,...,...
30,Colombia,CLUSTER CENTRO,CO-PLANTA CALLE 170,8012360524,2022-02-22,NaN,6.5,Facturado no despachado,Cancelado-Reusado
31,Colombia,CLUSTER CENTRO,CO-PLANTA SUR (DTE),8012363044,2022-02-23,NaN,7.0,Facturado no despachado,Cancelado-Desperdicio
32,Colombia,CLUSTER CENTRO,CO-PLANTA 240,8012363498,2022-02-23,NaN,6.5,Facturado no despachado,Cancelado-Reusado
33,Colombia,CLUSTER CENTRO,CO-PLANTA CALLE 170,8012371447,2022-02-25,NaN,5.0,Facturado no despachado,Cancelado-Desperdicio


In [6]:
params = urllib.parse.quote_plus("DRIVER={ODBC Driver 17 for SQL Server};SERVER=USCLDBITVMP01;DATABASE=BI_Tableau;UID=usertableau;PWD=usertableau$")
engine = sa.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)
        
df_sql.to_sql("SCAC_AT38_venta_vs_despacho", engine, index=False, if_exists="replace", schema="dbo")
        

# Despachados -> No Facturados

In [ ]:
df_final[df_final['Venta'].isnull()]

In [ ]:
df_final[df_final['Venta'].isnull()]['Despachado'].sum()

# Despachado != Facturado

In [ ]:
despacho_distinto_facturado.head(30)

In [ ]:
despacho_distinto_facturado[despacho_distinto_facturado['gap'] > 0]

In [ ]:
despacho_distinto_facturado[despacho_distinto_facturado['gap'] > 0]['gap'].sum()

# Facturado -> No despachado

In [ ]:
df_final2['Total'].sum()

In [ ]:
facturado_no_despachado

In [ ]:
facturado_no_despachado['Venta'].sum()

# Facturas a favor del cliente (valores negativos)

In [7]:
info_ventas[info_ventas['Total'] < 0]['Total'].sum()

-502.25